# 从零打造一个深度学习框架 （一）

华东师范大学 数学科学学院 陈久宁 2019年12月11日

这里讲解如何从作业及quiz中的代码结构演变出pytorch风格的代码结构

In [ ]:
from sklearn import datasets
import numpy as np
import math
import matplotlib.pyplot as plt

In [ ]:
def load_boston(ratio=0.8):
    X, Y = datasets.load_boston(True)
    Y.shape = -1, 1
    
    # normalization
    X = X/80
    Y = Y/(np.max(Y) - np.min(Y))
    
    num_samples = len(Y)
    num_train = math.ceil(num_samples * ratio)
    
    # 随机打乱数据
    idx = np.random.permutation(np.arange(num_samples))
    traindata = X[idx[:num_train]], Y[idx[:num_train]]
    validdata = X[idx[num_train:]], Y[idx[num_train:]]
    
    return traindata, validdata

In [ ]:
(X_train, Y_train), (X_valid, Y_valid) = load_boston()

## Naive Framework -- quiz version

In [ ]:
class Linear():
    def __init__(self, in_features, out_features):
        self.W = 1e-2 * np .random.randn(out_features, in_features)
        self.b = 1e-2 * np.random.randn(out_features)
    
    def __call__(self, X):
        return self.forward(X)
    
    def forward(self, X):
        return X @ self.W.T + self.b

In [ ]:
class Sigmoid():
    def __call__(self, X):
        return self.forward(X)
    
    def forward(self, X):
        return 1/(1+np.exp(-X))

`MLP`

* 缺陷：由于`grad`包含了完整的梯度计算过程，无法修改网络的深度及内置的具体网络层
* 解决方案: 将`grad`的实现拆分到具体网络层的实现中 

In [ ]:
class MLP:
    def __init__(self, in_features, mid_features, out_features):
        self.f1 = Linear(in_features, mid_features)
        self.phi = Sigmoid()
        self.f1 = Linear(in_features, mid_features)
        self.phi = Sigmoid()
        self.f2 = Linear(mid_features, out_features)
        
    def __call__(self, X):
        return self.f2(self.phi(self.f1(X)))
        
    def forward(self, X):
        Z1 = self.f1(X)
        Z2 = self.phi(Z1)
        Z3 = self.f2(Z2)
        return [X, Z1, Z2, Z3]
    
    def grad(self, Y, Z):
        Z0, Z1, Z2, Z3 = Z[0], Z[1], Z[2], Z[3]

        # Loss
        # Input: Z3, Y
        # Output: dLdZ3
        dLdZ3 = Z3 - Y

        # Linear -- f2
        # Input: dLdZ3, Z2
        # Output: dLdZ2, dLdW2, dLdb2
        dLdW2 = dLdZ3.T @ Z2
        dLdb2 = np.sum(dLdZ3, axis=0)
        dLdZ2 = dLdZ3 @ model.f2.W

        # Sigmoid
        # Input: dLdZ2, Z1
        # Output: dLdZ1, 
        expZ1 = np.exp(Z1)
        dLdZ1 = dLdZ2 * (expZ1/(np.exp(2*Z1)+2*expZ1+1))

        # Linear -- f1
        # Input: dLdZ1, Z0
        # Output: _, dLdW1, dLdb1
        dLdW1 = dLdZ1.T @ Z0
        dLdb1 = np.sum(dLdZ1, axis=0)

        return dLdW1, dLdb1, dLdW2, dLdb2

In [ ]:
def loss(Y_real, Y_pred):
    return 0.5 * np.sum((Y_real - Y_pred)**2)

`GradientDescent`

缺陷：不能支持其他类型的MLP —— 改变网络层数或者某一层的定义

解决方案：在每个网络层中定义一个函数来提取当前网络层的所有权重，然后在Optimizer中通过该函数来获取信息

In [ ]:
class GradientDescent:
    def __init__(self, step=1e-3):
        self.step = step
        
    def update(self, model:MLP, dLdW1, dLdb1, dLdW2, dLdb2):
        """利用梯度dW来更新f的权重"""
        # f1, f2, W, b跟具体层数和网络的实现有关
        model.f2.W -= self.step * dLdW2
        model.f2.b -= self.step * dLdb2
        
        model.f1.W -= self.step * dLdW1
        model.f1.b -= self.step * dLdb1

## Naive framework -- torch-like version

接下来实现一个可以一般化的网络设计，具体要做的事情为：

1. 将`grad`的计算拆分到具体网络层及Loss中，而不是放在`MLP`里
2. 在每个具体网络层中添加一个函数，来返回当前网络层所对应的权重矩阵
3. 利用循环来迭代前向、反向及梯度更新


In [ ]:
class Linear():
    def __init__(self, in_features, out_features):
        self.W = 1e-2 * np .random.randn(out_features, in_features)
        self.b = 1e-2 * np.random.randn(out_features)
    
    def __call__(self, X):
        return self.forward(X)
    
    def forward(self, X):
        return X @ self.W.T + self.b
    
    def backward(self, dLdZ, X):
        dLdW = dLdZ.T @ X
        dLdb = np.sum(dLdZ, axis=0)
        dLdX = dLdZ @ self.W
        
        assert dLdW.shape == self.W.shape
        assert dLdb.shape == self.b.shape
        assert dLdX.shape == X.shape
        
        return dLdX, (dLdW, dLdb)
    
    @property
    def parameters(self):
        return self.W, self.b

In [ ]:
class Sigmoid():
    def __call__(self, X):
        return self.forward(X)
    
    def forward(self, X):
        return 1/(1+np.exp(-X))
    
    def backward(self, dLdZ, X):
        expX = np.exp(X)
        dLdX = dLdZ * (expX/(np.exp(2*X)+2*expX+1))
        
        assert dLdX.shape == X.shape
        
        return dLdX, None
    
    @property
    def parameters(self):
        return None

将loss的计算及梯度的计算放在类里

In [ ]:
class MSELoss():
    def __call__(self, *args):
        return self.forward(*args)
    
    def forward(self, pred, label):
        return 0.5 * np.sum((pred - label)**2)
    
    def backward(self, label, X):
        dLdX = X - label
        
        assert dLdX.shape == X.shape
        return dLdX

构造一个meta model

In [ ]:
class Sequential:
    def __init__(self, *layers):
        self.layers = layers
    
    def __call__(self, X):
        return self.forward(X)[-1]
        
    def forward(self, X):
        forward_results = [X, ]
        Z = X
        for layer in self.layers:
            Z = layer(Z)
            forward_results.append(Z)
        
        return forward_results
    
    def backward(self, dL, X):
        Z = self.forward(X)
        
        parameter_grads = []
        dLdZ = dL
        for layer, X in zip(reversed(self.layers),
                            reversed(Z[0:-1])):
            dLdZ, dLdP = layer.backward(dLdZ, X)
            if dLdP:
                parameter_grads.extend(reversed(dLdP))

        assert dLdZ.shape == X.shape
        return dLdZ, list(reversed(parameter_grads))
    
    @property
    def parameters(self):
        params = []
        for layer in self.layers:
            p = layer.parameters
            if p:
                params.extend(p)
        return params

In [ ]:
model = Sequential(
    Linear(13, 50),
    Sigmoid(),
    Linear(50, 1)
)

loss = MSELoss()

Z = model.forward(X_train)
l = loss(Y_train, Z[-1])
dL = loss.backward(Y_train, Z[-1])
dLdX, dLdP = model.backward(dL, X_train)

print(dLdX.shape)
print(len(dLdP))
assert dLdP[0].shape == model.parameters[0].shape
assert dLdP[1].shape == model.parameters[1].shape

权重与权重的梯度对应之后，就可以开始写Optimizer了

In [ ]:
class GradientDescent:
    def __init__(self, params, step=1e-3):
        self.params = params
        self.step = step
        
    def update(self, dLdParams):
        for p, dLdp in zip(self.params, dLdParams):
            p -= self.step * dLdp

训练网络

In [ ]:
num_features = X_train.shape[-1]
model = Sequential(
    Linear(num_features, 50),
    Sigmoid(),
    Linear(50, 10),
    Sigmoid(),
    Linear(10, 1)
)
loss = MSELoss()
opt = GradientDescent(model.parameters, 1e-5)

valid_losses = []
train_losses = []
for i in range(1000):
    X, Y = X_train, Y_train
    
    Z = model.forward(X)
    l = loss(Y, Z[-1])
    dL = loss.backward(Y, Z[-1])
    _, dLdP = model.backward(dL, X)
    opt.update(dLdP)
    
    Y_out = Z[-1]
    cur_valid_loss = loss(Y_valid, model(X_valid))
    cur_train_loss = loss(Y, Y_out)
    valid_losses.append(cur_valid_loss) 
    train_losses.append(cur_train_loss)
    
    if i%100 == 0:
        print(f"Iter {i}: loss {cur_train_loss:.4f}, valid loss {cur_valid_loss:.4f}")

In [ ]:
plt.plot(train_losses)
plt.plot(valid_losses)
plt.legend(["train loss", "validation loss"])

### What's after?

* GPU CUDA wrapper -- 如何调用显卡
* memory reduce -- 如何减少不必要的内存开销
* autograd -- 如何进行导数的自动推断，从而隐藏backward实现